Домашняя работа
Взять boston house-prices datase (sklearn.datasets.load_boston). Возмите 7 любых регрессоров (попробовать разные алгоритмы, поподбирать параметры, вывести итоговое качество).

In [15]:
import numpy as np
import operator
import pandas as pd # Для работы с данными
import scipy.stats # При работе со статистикой
from scipy.stats import uniform 
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import RandomizedSearchCV

from sklearn.datasets import load_boston


In [6]:
boston_dataset = load_boston()

In [7]:
print(boston_dataset.keys())
#печатаем значение boston_dataset чтобы понять, что в нем содержится

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])


In [8]:
#загрузим данные в фрейм данных pandas, используя pd.DataFrame, Затем мы печатаем первые 5 строк данных, используя head()
boston = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [9]:
#целевое значение MEDV отсутствует в данных. Мы создаем новый столбец целевых значений и добавляем его в фрейм данных.
boston['MEDV'] = boston_dataset.target
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [10]:
from sklearn.pipeline import make_pipeline # используем пайплайны для удобства
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 

#model = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))

In [34]:
#выберем для X  три параметра boston['LSTAT'], boston['RM'], boston['PTRATIO']
X = pd.DataFrame(np.c_[boston['LSTAT'], boston['RM'], boston['PTRATIO']], columns = ['LSTAT','RM', 'PTRATIO'])
Y = boston['MEDV']

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=5)

In [14]:
from sklearn.linear_model import LinearRegression # линейная регрессия
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import GradientBoostingRegressor # Алгоритм дерева решений с градиентным ускорением

In [35]:
# сформируем список из словарей с моделями и их параметрами 
models=[
      {'name':'LR',"model":LinearRegression(), 'params':{'fit_intercept': ['True', 'False'], 'normalize':['True', 'False']}}, 
      {'name':'KN',"model": KNeighborsRegressor(), 'params':{'n_neighbors':list(range(1,30)),'weights': ['uniform', 'distance'], 'p':[1,2,3]}},
      {'name':'R',"model": Ridge(), 'params':{'alpha': uniform(loc=0, scale=4), 'solver':['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']}},
      {'name':'RF',"model": RandomForestRegressor(), 'params':{'n_estimators':[10,25,50,100], 'criterion':['mse', 'mae'], 'max_depth':[3,5,7,9,11]}},
      {'name':'DT',"model": DecisionTreeRegressor(), 'params':{'criterion':['mse', 'friedman_mse', 'mae', 'poisson'], 'max_depth':[3,5,7,9,11]}},
      {'name':'SGD',"model": SGDRegressor(), 'params':{'loss':['squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'], 'penalty':['l2', 'l1', 'elasticnet']}},
      {'name':'GDB',"model": GradientBoostingRegressor(), 'params':{'loss':['ls', 'lad', 'huber', 'quantile'], 'n_estimators':[10,25,50,100]}},
]

# передадим указанный список в RandomizedSearchCV для проведения анализа гиперпараметров
res=[]
for v in  models:    res.append((v['name'], RandomizedSearchCV(v['model'], v['params'], cv=10).fit(X_train, Y_train)))


In [36]:
for r in res:
    print(r[0], r[1].best_score_, r[1].best_params_)

LR 0.6542993604435722 {'normalize': 'True', 'fit_intercept': 'True'}
KN 0.7461826338425858 {'weights': 'uniform', 'p': 1, 'n_neighbors': 13}
R 0.6543006673730198 {'alpha': 1.064379290794757, 'solver': 'sparse_cg'}
RF 0.7804646292333313 {'n_estimators': 100, 'max_depth': 5, 'criterion': 'mse'}
DT 0.7254384995718667 {'max_depth': 5, 'criterion': 'mae'}
SGD 0.6349737673418032 {'penalty': 'l1', 'loss': 'epsilon_insensitive'}
GDB 0.8027757734861624 {'n_estimators': 50, 'loss': 'huber'}


Видим, значения параметор для каждого из алгоритма. Обращаем особое внимание на существенно лучший результат у алгоритма  дерева решений с градиентным ускорением GDB 0.8027757734861624 со значением параметров {'n_estimators': 50, 'loss': 'huber'}